In [0]:
# Creacion de los schemas y tablas requeridos
dbutils.widgets.removeAll()

In [0]:
schemas_list = ["bronze","silver","golden"]

dbutils.widgets.text("catalog", "dev_ecommerce")
dbutils.widgets.text("storage_account","adlsproyecto2025")


In [0]:
catalog_param = dbutils.widgets.get("catalog")
storage_account = dbutils.widgets.get("storage_account")

In [0]:
def delete_folders(path):
    try:
        dbutils.fs.rm(path, recurse=True)
        print(f"Folder {path} eliminado.")
    except Exception:
        print(f"No existe folder: {path}")
        pass
    
def create_schema(schema):
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_param}.{schema}")

def drop_schema(schema):
    spark.sql(f"DROP SCHEMA IF EXISTS {catalog_param}.{schema} CASCADE")


def use_catalog():
    catalogs = [row.catalog for row in spark.sql("SHOW CATALOGS").collect()]
    
    if catalog_param in catalogs:
        spark.sql(f"USE CATALOG {catalog_param}")
    else:
        print(f"El catálogo '{catalog_param}' aun no existe.")

In [0]:
# seleccion del catalogo
use_catalog()

# Limpieza de tablas
for schema in schemas_list:
    drop_schema(schema)

# Eliminacion del catalogo
spark.sql(f" DROP CATALOG IF EXISTS {catalog_param} CASCADE")

In [0]:
# Eliminar todas las carpetas de los containers

#dbutils.fs.rm("abfss://bronze@adlsproyecto2025.dfs.core.windows.net/sales_orders/", recurse=True)

for container in schemas_list:
    delete_folders(f"abfss://{container}@{storage_account}.dfs.core.windows.net/")




In [0]:
# crea el catálogo
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_param}")

# seleccion del catalogo
use_catalog()


In [0]:
# crea los schemas 
for schema in schemas_list:
    create_schema(schema)

In [0]:

print(f"Se ha creado el catálogo '{catalog_param}' y los esquemas")
      